In [1]:
from src.task_3 import main

In [2]:
main()

Pipeline execution completed.


# Overview

<div style="text-align: center;">
  <img src="./docs/pipeline.png" alt="Docs" width="40%"/>
</div>

The figure above describes the steps taken by the `main` function. The pipeline consists of three blocks:

- `DataLoader` handles data-related tasks such as loading images and saving CSV files.
- `ModelRunner` manages the computer vision models required in the pipeline.
- `ReportGenerator` conducts statistical analysis on the results and generates a report document (output.md).

### Detailed Breakdown

Using these three classes, the pipeline can be broken down as follows:

1. **Data Loading**:
    - `DataLoader` calls `get_wsi_paths()` to find the WSI paths inside a specified parent directory using `os.listdir` and other methods.
    - For each WSI path:
        1. The `DataLoader`'s `load_wsi` method loads the WSI image into memory.
        2. The `DataLoader`'s `extract_patches` method extracts all patches of a given size from the image, filtering out mainly white images using a pixel mean threshold.

2. **Model Running**:
    - The `ModelRunner`'s `run_segmentation` method runs segmentation on all the patches. Ideally, a finetuned *Cellpose* model trained on the specific dataset and image style would be used.
    - The `ModelRunner`'s `run_classification` method is applied to each segment. This could involve cropping the image around each bounding box containing a segment and classifying the contents. A well-trained convolutional classification architecture would be suitable. Alternatively, a semantic segmentation model could be designed to replace the segmentation/classification model pair and perform direct classification. However, this approach involves more complex modeling due to the increased task difficulty.
    - The results of the classification are saved as a `.csv` file by passing the results of the `run_classification` method to the `DataLoader`'s `save_classification_results` method.

3. **Report Generation**:
    - Assuming each WSI path is linked to a `patient_id`, experiment `arm`, and `treatment` category, statistical analysis is run on a merged dataset containing all the `.csv` files from the previous step. This generates metrics seen in [task_2.ipynb](./task_2.ipynb), including p-tests for the null hypothesis.
    - The results of the p-tests are then passed to either an LLM API such as ChatGPT 3.5 using the `openai` Python library, or a local instance of an LLM such as Llama 3 8B instruct running on a *Hugging Face* `pipeline` or *Ollama*'s *LangChain* import. A detailed prompt instructs the LLM on how to format and write the report, given the results of the statistical analysis.
    - This report is saved locally to a desired output directory. Alternatively, it could be automatically emailed to all team members or uploaded to a company shared file storage (such as Azure or a shared folder in Slack).